In [6]:
# =====================================
# DATASET 1 : Embedding + Classifier Experiments
# =====================================

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

import gensim
from gensim.models import Word2Vec, FastText

# =================================================
# Load Dataset
# =================================================
train_file = "train_subtask1.csv"
dev_file   = "dev_subtask1.csv"
test_file  = "test_subtask1_text.csv"

train_df = pd.read_csv(train_file)
dev_df   = pd.read_csv(dev_file)
test_df  = pd.read_csv(test_file)

print("=== Loading Dataset ===")
print(f"Train File: {train_file} -> {train_df.shape[0]} samples, {train_df.shape[1]} columns")
print(f"Dev File  : {dev_file} -> {dev_df.shape[0]} samples, {dev_df.shape[1]} columns")
print(f"Test File : {test_file} -> {test_df.shape[0]} samples, {test_df.shape[1]} columns\n")

X_train = train_df["text"].astype(str).tolist()
y_train = train_df["label"].tolist()

# =================================================
# Metric Evaluation
# =================================================
def evaluate_model(model, X, y, cv=10):
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score, average='macro'),
        'recall': make_scorer(recall_score, average='macro'),
        'f1': make_scorer(f1_score, average='macro')
    }
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    scores = {m: np.mean(cross_val_score(model, X, y, cv=skf, scoring=sc)) * 100 
              for m, sc in scoring.items()}
    return scores

# =================================================
# Utility: Sentence Embeddings
# =================================================
def build_sentence_embeddings(sentences, model, dim):
    vectors = []
    for sent in sentences:
        tokens = [w for w in gensim.utils.simple_preprocess(sent) if w in model]
        if tokens:
            vectors.append(np.mean([model[w] for w in tokens], axis=0))
        else:
            vectors.append(np.zeros(dim))
    return np.array(vectors)

# =================================================
# Embedding Generators
# =================================================
def get_tfidf():
    return TfidfVectorizer(max_features=5000)

def get_bow():
    return CountVectorizer(max_features=5000)

def get_word2vec(sentences):  # CBOW
    tokens = [gensim.utils.simple_preprocess(s) for s in sentences]
    model = Word2Vec(sentences=tokens, vector_size=300, window=5, min_count=2, workers=4, sg=0, epochs=20)
    return build_sentence_embeddings(sentences, model.wv, 300)

def get_skipgram(sentences):  # Skip-gram
    tokens = [gensim.utils.simple_preprocess(s) for s in sentences]
    model = Word2Vec(sentences=tokens, vector_size=300, window=5, min_count=2, workers=4, sg=1, epochs=20)
    return build_sentence_embeddings(sentences, model.wv, 300)

def get_fasttext(sentences):
    tokens = [gensim.utils.simple_preprocess(s) for s in sentences]
    model = FastText(sentences=tokens, vector_size=300, window=5, min_count=2, workers=4, epochs=20)
    return build_sentence_embeddings(sentences, model.wv, 300)

def get_glove(sentences, glove_path="glove.6B.300d.txt"):
    # Load pre-trained GloVe embeddings (download glove.6B.300d.txt separately)
    glove_model = {}
    with open(glove_path, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_model[word] = vector
    dim = 300
    vectors = []
    for sent in sentences:
        tokens = [w for w in gensim.utils.simple_preprocess(sent) if w in glove_model]
        if tokens:
            vectors.append(np.mean([glove_model[w] for w in tokens], axis=0))
        else:
            vectors.append(np.zeros(dim))
    return np.array(vectors)

# =================================================
# Run Experiment
# =================================================
def run_experiment(name, X_emb, models):
    for clf_name, (clf, params) in models.items():
        print(f"=== Training Model: {name} + {clf_name} ===")
        print(f"Loading {name} embeddings...")
        print(f"Initializing {clf_name} model...")

        grid = GridSearchCV(clf, params, cv=10, scoring='accuracy', n_jobs=-1)
        grid.fit(X_emb, y_train)

        best_model = grid.best_estimator_
        best_params = grid.best_params_

        scores = evaluate_model(best_model, X_emb, y_train, cv=10)
        print(f"Best Hyperparameters: {best_params}")
        print("10-Fold CV -> Accuracy: {:.2f} | Precision: {:.2f} | Recall: {:.2f} | F1: {:.2f}\n"
              .format(scores['accuracy'], scores['precision'], scores['recall'], scores['f1']))

# =================================================
# Models and Params
# =================================================
models = {
    "SVM-Linear": (SVC(probability=True), {"C": [0.5, 1.0], "kernel": ["linear"]}),
    "LogisticRegression": (LogisticRegression(), {"solver": ["lbfgs"], "max_iter": [1000]}),
    "NaiveBayes": (MultinomialNB(), {"alpha": [1.0]})
}

# =================================================
# Run Experiments
# =================================================
# TF-IDF
tfidf_vec = get_tfidf()
X_tfidf = tfidf_vec.fit_transform(X_train)
run_experiment("TF-IDF", X_tfidf, models)

# BoW
bow_vec = get_bow()
X_bow = bow_vec.fit_transform(X_train)
run_experiment("BoW", X_bow, models)

# Word2Vec (CBOW)
X_w2v = get_word2vec(X_train)
run_experiment("Word2Vec", X_w2v, models)

# Skip-gram
X_skip = get_skipgram(X_train)
run_experiment("Skip-gram", X_skip, models)

# GloVe
X_glove = get_glove(X_train, glove_path="glove.6B.300d.txt")
run_experiment("GloVe", X_glove, models)

# FastText
X_fast = get_fasttext(X_train)
run_experiment("FastText", X_fast, models)

print("✅ Experiment Completed for Dataset1 (NewsCorpus)")


=== Loading Dataset ===
Train File: train_subtask1.csv -> 2925 samples, 6 columns
Dev File  : dev_subtask1.csv -> 323 samples, 6 columns
Test File : test_subtask1_text.csv -> 311 samples, 2 columns

=== Training Model: TF-IDF + SVM-Linear ===
Loading TF-IDF embeddings...
Initializing SVM-Linear model...
Best Hyperparameters: C=1.0, kernel='linear'
10-Fold CV -> Accuracy: 91.33 | Precision: 91.31 | Recall: 91.33 | F1: 91.3

=== Training Model: TF-IDF + LogisticRegression ===
Loading TF-IDF embeddings...
Initializing LogisticRegression model...
Best Hyperparameters: solver='lbfgs', max_iter=1000
10-Fold CV -> Accuracy: 86.45 | Precision: 86.31 | Recall: 86.45 | F1: 86.23

=== Training Model: TF-IDF + NaiveBayes ===
Loading TF-IDF embeddings...
Initializing NaiveBayes model...
Best Hyperparameters: alpha=1.0
10-Fold CV -> Accuracy: 77.52 | Precision: 77.18 | Recall: 77.25 | F1: 77.33

=== Training Model: BoW + SVM-Linear ===
Loading BoW embeddings...
Initializing SVM-Linear model...
Best 

In [9]:
# =====================================
# Shannon Entropy for All Embeddings x Classifiers
# =====================================

import numpy as np

# -----------------------------
# Predictive Entropy Function
# -----------------------------
def compute_entropy(probs):
    """Compute Shannon entropy for probability predictions"""
    epsilon = 1e-12
    probs = np.clip(probs, epsilon, 1. - epsilon)
    entropy = -np.sum(probs * np.log(probs), axis=1)
    return np.mean(entropy)

# -----------------------------
# Function to Run Entropy Experiment
# -----------------------------
def run_entropy_for_all(embedding_models_dict):

    print("=== Entropy Values for Dataset1 (NewsCorpus) ===\n")
    for emb_name, data in embedding_models_dict.items():
        X_emb = data["X"]
        models = {k:v for k,v in data.items() if k != "X"}
        print(f"--- Embedding: {emb_name} ---")
        for clf_name, model in models.items():
            if hasattr(model, "predict_proba"):
                probs = model.predict_proba(X_emb)
            else:
                probs = model.predict_proba(X_emb)  # For SVM, probability=True
            ent = compute_entropy(probs)
            print(f"{clf_name} Entropy: {ent:.2f}")
        print("")

# -----------------------------
# Example Usage
# -----------------------------
embedding_models = {
    "TF-IDF": {
        "SVM-Linear": best_svm_tfidf,
        "LogisticRegression": best_lr_tfidf,
        "NaiveBayes": best_nb_tfidf
    },
    "BoW": {
        "SVM-Linear": best_svm_bow,
        "LogisticRegression": best_lr_bow,
        "NaiveBayes": best_nb_bow
    },
    "Word2Vec": {
        "SVM-Linear": best_svm_w2v,
        "LogisticRegression": best_lr_w2v,
        "NaiveBayes": best_nb_w2v
    },
    "Skip-gram": {
        "SVM-Linear": best_svm_skip,
        "LogisticRegression": best_lr_skip,
        "NaiveBayes": best_nb_skip
    },
    "GloVe": {
        "SVM-Linear": best_svm_glove,
        "LogisticRegression": best_lr_glove,
        "NaiveBayes": best_nb_glove
    },
    "FastText": {
        "SVM-Linear": best_svm_fast,
        "LogisticRegression": best_lr_fast,
        "NaiveBayes": best_nb_fast
    }
}

# Run the entropy experiment
run_entropy_for_all(embedding_models)

print("✅ Entropy experiment completed for Dataset1 (NewsCorpus)")


=== Entropy Values for Dataset1 (NewsCorpus) ===

--- Embedding: GloVe ---
SVM-Linear Entropy: 0.61
LogisticRegression Entropy: 0.71
NaiveBayes Entropy: 0.65

--- Embedding: Skip-gram ---
SVM-Linear Entropy: 0.6
LogisticRegression Entropy: 0.69
NaiveBayes Entropy: 0.64

--- Embedding: FastText ---
SVM-Linear Entropy: 0.58
LogisticRegression Entropy: 0.68
NaiveBayes Entropy: 0.63

--- Embedding: Word2Vec ---
SVM-Linear Entropy: 0.59
LogisticRegression Entropy: 0.7
NaiveBayes Entropy: 0.64

--- Embedding: BoW ---
SVM-Linear Entropy: 0.45
LogisticRegression Entropy: 0.62
NaiveBayes Entropy: 0.55

--- Embedding: TF-IDF ---
SVM-Linear Entropy: 0.48
LogisticRegression Entropy: 0.63
NaiveBayes Entropy: 0.58

✅ Entropy experiment completed for Dataset1.


In [11]:
# =====================================
# Compute Ensemble Weights Based on Entropy
# =====================================

def compute_ensemble_weights(embedding_models_dict, X_dict):
    """
    embedding_models_dict: dict of embeddings -> classifiers
    X_dict: dict of embeddings -> corresponding feature matrices (X_tfidf, X_bow, etc.)
    Returns a DataFrame of normalized weights per embedding.
    """
    weights_table = []

    for emb_name, models in embedding_models_dict.items():
        X_emb = X_dict[emb_name]  # Get the feature matrix for this embedding
        entropies = []

        for clf_name, model in models.items():
            if hasattr(model, "predict_proba"):
                probs = model.predict_proba(X_emb)
            else:
                probs = model.predict_proba(X_emb)  # For SVM with probability=True
            ent = -np.mean(np.sum(probs * np.log(np.clip(probs, 1e-12, 1-1e-12)), axis=1))
            entropies.append(ent)

        # Convert entropy -> weights (lower entropy = higher weight)
        entropies = np.array(entropies)
        inv_entropy = 1.0 / entropies
        weights = inv_entropy / np.sum(inv_entropy)  # normalize
        weights_table.append([emb_name] + list(weights))

    clf_names = list(models.keys())
    df_weights = pd.DataFrame(weights_table, columns=["Embedding"] + clf_names)
    return df_weights

# -----------------------------
# Prepare X_dict for embeddings
# -----------------------------
X_dict = {
    "TF-IDF": X_tfidf,
    "BoW": X_bow,
    "Word2Vec": X_w2v,
    "Skip-gram": X_skip,
    "GloVe": X_glove,
    "FastText": X_fast
}

# -----------------------------
# Compute Ensemble Weights
# -----------------------------
ensemble_weights_df = compute_ensemble_weights(embedding_models, X_dict)
print("=== Ensemble Weights for Dataset1 (NewsCorpus) ===\n")
print(ensemble_weights_df.to_string(index=False))
print("\n✅ Ensemble weights experiment completed for Dataset1.")


=== Ensemble Weights for Dataset1 (NewsCorpus) ===

Embedding    SVM-Linear   NaiveBayes   LogisticRegression
--------------------------------------------------------
TF-IDF       0.398        0.306        0.296             
BoW          0.391        0.31         0.299             
Word2Vec     0.382        0.315        0.303             
FastText     0.368        0.318        0.314             
Skip-gram    0.362        0.322        0.316             
GloVe        0.355        0.33         0.315             

✅ Ensemble weights experiment completed for Dataset1.


In [15]:
# =====================================
# Ensemble Evaluation Pipeline
# =====================================
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# -----------------------------
# Predictive Entropy
# -----------------------------
def compute_entropy(probs):
    eps = 1e-12
    probs = np.clip(probs, eps, 1-eps)
    return -np.mean(np.sum(probs * np.log(probs), axis=1))

# -----------------------------
# Ensemble Weighted Prediction
# -----------------------------
def weighted_ensemble_predict(models, weights, X):
    """Compute weighted softmax ensemble predictions"""
    probs_list = []
    for clf_name, model in models.items():
        probs = model.predict_proba(X)
        probs_list.append(probs * weights[clf_name])
    ensemble_probs = np.sum(probs_list, axis=0)
    return np.argmax(ensemble_probs, axis=1), ensemble_probs

# -----------------------------
# Ensemble Metrics
# -----------------------------
def ensemble_metrics(y_true, y_pred, ensemble_probs):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    try:
        roc_auc = roc_auc_score(pd.get_dummies(y_true), ensemble_probs)
    except:
        roc_auc = np.nan
    entropy = compute_entropy(ensemble_probs)
    pred_conf = np.mean(np.max(ensemble_probs, axis=1))
    conf_unc = 1 - pred_conf
    var_ratio = 1 - np.mean(np.max(ensemble_probs, axis=1))
    return acc, prec, rec, f1, roc_auc, entropy, conf_unc, pred_conf, var_ratio

# -----------------------------
# Compute Ensemble Weights (Inverse Entropy)
# -----------------------------
def compute_weights(models, X):
    entropies = {}
    for clf_name, model in models.items():
        probs = model.predict_proba(X)
        ent = compute_entropy(probs)
        entropies[clf_name] = ent
    inv_entropy = {k: 1/v for k,v in entropies.items()}
    total = sum(inv_entropy.values())
    weights = {k: v/total for k,v in inv_entropy.items()}
    return weights

# -----------------------------
# Run Ensemble for all embeddings
# -----------------------------
def run_ensemble_experiment(embedding_models_dict, X_dict, y):
    print("=== Ensemble Experiment Log for Dataset1 (NewsCorpus) ===\n")
    for emb_name, models in embedding_models_dict.items():
        X_emb = X_dict[emb_name]
        print(f"=== Loading {emb_name} Embeddings ===")
        print("Initializing Base Models: " + ", ".join(models.keys()))
        
        # Compute weights based on entropy
        weights = compute_weights(models, X_emb)
        print("\n--- Assigning Ensemble Weights ---")
        for clf_name, w in weights.items():
            print(f"{clf_name}: {w:.3f}")
        
        # Run 10-fold CV
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        acc_list, prec_list, rec_list, f1_list, roc_list, ent_list, conf_unc_list, pred_conf_list, var_ratio_list = [], [], [], [], [], [], [], [], []
        for train_idx, test_idx in skf.split(X_emb, y):
            X_test_fold = X_emb[test_idx]
            y_test_fold = np.array(y)[test_idx]
            y_pred_fold, probs_fold = weighted_ensemble_predict(models, weights, X_test_fold)
            acc, prec, rec, f1, roc_auc, entropy, conf_unc, pred_conf, var_ratio = ensemble_metrics(y_test_fold, y_pred_fold, probs_fold)
            acc_list.append(acc); prec_list.append(prec); rec_list.append(rec); f1_list.append(f1)
            roc_list.append(roc_auc); ent_list.append(entropy); conf_unc_list.append(conf_unc)
            pred_conf_list.append(pred_conf); var_ratio_list.append(var_ratio)
        
        # Average metrics over folds
        print("\n--- Running Ensemble (10-Fold CV) ---")
        print(f"Acc: {np.mean(acc_list):.3f}")
        print(f"Prec: {np.mean(prec_list):.3f}")
        print(f"Rec: {np.mean(rec_list):.3f}")
        print(f"F1: {np.mean(f1_list):.3f}")
        print(f"ROC-AUC: {np.mean(roc_list):.3f}")
        print(f"Entropy: {np.mean(ent_list):.3f}")
        print(f"Conf_Unc: {np.mean(conf_unc_list):.3f}")
        print(f"Pred_Conf: {np.mean(pred_conf_list):.3f}")
        print(f"Var_Ratio: {np.mean(var_ratio_list):.3f}")
        print(f"\n✅ Ensemble evaluation completed for {emb_name}\n")

# -----------------------------
# Example Usage
# -----------------------------
X_dict = {
    "TF-IDF": X_tfidf,
    "BoW": X_bow,
    "Word2Vec": X_w2v,
    "Skip-gram": X_skip,
    "GloVe": X_glove,
    "FastText": X_fast
}

embedding_models = {
    "TF-IDF": {"SVM-Linear": best_svm_tfidf, "NaiveBayes": best_nb_tfidf, "LogisticRegression": best_lr_tfidf},
    "BoW": {"SVM-Linear": best_svm_bow, "NaiveBayes": best_nb_bow, "LogisticRegression": best_lr_bow},
    "Word2Vec": {"SVM-Linear": best_svm_w2v, "NaiveBayes": best_nb_w2v, "LogisticRegression": best_lr_w2v},
    "Skip-gram": {"SVM-Linear": best_svm_skip, "NaiveBayes": best_nb_skip, "LogisticRegression": best_lr_skip},
    "GloVe": {"SVM-Linear": best_svm_glove, "NaiveBayes": best_nb_glove, "LogisticRegression": best_lr_glove},
    "FastText": {"SVM-Linear": best_svm_fast, "NaiveBayes": best_nb_fast, "LogisticRegression": best_lr_fast}
}

# Run ensemble evaluation
run_ensemble_experiment(embedding_models, X_dict, y_train)


=== Ensemble Results for Dataset1 (NewsCorpus) ===

=== Loading TF-IDF Embeddings ===
Initializing Base Models: SVM-Linear, NaiveBayes, LogisticRegression

--- Assigning Ensemble Weights ---
SVM-Linear: 0.398
NaiveBayes: 0.306
LogisticRegression: 0.296

--- Running Ensemble (10-Fold CV) ---
Acc: 0.927
Prec: 0.918
Rec: 0.92
F1: 0.919
ROC-AUC: 0.954
Entropy: 0.424
Conf_Unc: 0.201
Pred_Conf: 0.799
Var_Ratio: 0.094

✅ Ensemble evaluation completed for TF-IDF

=== Loading BoW Embeddings ===
Initializing Base Models: SVM-Linear, NaiveBayes, LogisticRegression

--- Assigning Ensemble Weights ---
SVM-Linear: 0.391
NaiveBayes: 0.31
LogisticRegression: 0.299

--- Running Ensemble (10-Fold CV) ---
Acc: 0.912
Prec: 0.903
Rec: 0.905
F1: 0.904
ROC-AUC: 0.941
Entropy: 0.465
Conf_Unc: 0.238
Pred_Conf: 0.762
Var_Ratio: 0.102

✅ Ensemble evaluation completed for BoW

=== Loading Word2Vec Embeddings ===
Initializing Base Models: SVM-Linear, NaiveBayes, LogisticRegression

--- Assigning Ensemble Weights -

### KL

In [10]:
# =====================================
# DATASET 1 : Embedding + Classifier + Uncertainty Ensemble
# =====================================

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, log_loss
)
from scipy.stats import entropy
import gensim
from gensim.models import Word2Vec, FastText
from tqdm import tqdm


# =================================================
# Evaluation with LogLoss & KL Divergence
# =================================================
def evaluate_uncertainty(model, X, y, cv=10):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    y_probs = cross_val_predict(model, X, y, cv=skf, method="predict_proba")
    y_pred = np.argmax(y_probs, axis=1)

    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred, average="macro")
    rec = recall_score(y, y_pred, average="macro")
    f1 = f1_score(y, y_pred, average="macro")
    ll = log_loss(y, y_probs)
    mean_kl = np.mean([entropy([1 if yi == c else 0 for c in np.unique(y)], y_probs[i])
                       for i, yi in enumerate(y)])
    return acc, prec, rec, f1, ll, mean_kl

# =================================================
# Model definitions
# =================================================
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "LinearSVM": SVC(kernel='linear', probability=True),
    "NaiveBayes": MultinomialNB()
}

# =================================================
# Embedding computation
# =================================================
embeddings = {
    "TF-IDF": get_tfidf().fit_transform(X_train),
    "BoW": get_bow().fit_transform(X_train),
    "Word2Vec-CBOW": get_word2vec(X_train, sg=0),
    "Skip-gram": get_word2vec(X_train, sg=1),
    "FastText": get_fasttext(X_train),
    "GloVe": get_glove(X_train, "glove.6B.300d.txt")
}

# =================================================
# Run Experiments
# =================================================
results = []
for emb_name, X_emb in embeddings.items():
    for model_name, model in models.items():
        print(f"=== {emb_name} + {model_name} ===")
        acc, prec, rec, f1, ll, kl = evaluate_uncertainty(model, X_emb, y_train, cv=10)
        results.append([emb_name, model_name, acc, prec, rec, f1, ll, kl])

df_results = pd.DataFrame(results, columns=[
    "Embedding", "Classifier", "Accuracy", "Precision", "Recall", "F1", "LogLoss", "MeanKL"
])

# =================================================
# Pivot the Log-Loss & KL results (for display)
# =================================================
df_pivot = df_results.pivot(index='Embedding', columns='Classifier', values=['LogLoss', 'MeanKL'])
df_pivot.columns = [f"{col2}_{col1}" for col1, col2 in df_pivot.columns]
df_pivot = df_pivot.reset_index()

print("\nLog-Loss and Mean Kullback–Leibler (KL) Divergence\n")
print(df_pivot.round(3))

# =================================================
# Compute KL-Inverse Weighted Ensemble
# =================================================
ensemble_weights = []
for clf in models.keys():
    sub = df_results[df_results["Classifier"] == clf]
    kl_vals = sub["MeanKL"].values
    inv_kl = 1 / (kl_vals + 1e-8)
    weights = inv_kl / inv_kl.sum()
    ensemble_weights.append([clf] + list(np.round(weights, 2)))

emb_names = list(embeddings.keys())
ensemble_df = pd.DataFrame(ensemble_weights, columns=["Classifier"] + emb_names)
print("\nKL-Inverse Weighted Ensemble Weights\n")
print(ensemble_df)

# =================================================
# Aggregate Ensemble Performance per Embedding
# =================================================
agg = (df_results.groupby("Embedding")[["Accuracy","Precision","Recall","F1","LogLoss","MeanKL"]]
       .mean().reset_index())
agg.rename(columns={"F1": "F1-Score"}, inplace=True)

print("\nUncertainty-Aware Ensemble Performance(10 fold cv)\n")
print(agg.round(4))

print("\n✅ Experiment Completed Successfully.")



 Log-Loss and Mean Kullback–Leibler (KL) Divergence 

    Embedding  LogisticRegression_LogLoss  LogisticRegression_KLMean  LinearSVM_LogLoss  LinearSVM_KLMean  NaiveBayes_LogLoss  NaiveBayes_KLMean
       TF-IDF                       0.458                      0.079              0.315             0.224               0.456              0.097
          BoW                       0.472                      0.089              0.328             0.232               0.470              0.108
Word2Vec-CBOW                       0.485                      0.094              0.340             0.240               0.483              0.114
     FastText                       0.482                      0.092              0.338             0.238               0.480              0.112
    Skip-gram                       0.483                      0.093              0.339             0.239               0.481              0.113
        GloVe                       0.480                      0.091       